In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)


In [2]:
# plot a histogram
# remove negative ratings
# remove negative ratings

##### [RQ7] Estimating probabilities is a core skill for a data scientist: show us your best!

- Estimate the probability that a book has over 30% of the ratings above 4.
- Estimate the probability that an author publishes a new book within two years from its last work.
- In the file list.json, you will find a peculiar list named "The Worst Books of All Time." Estimate the probability of a book being included in this list, knowing it has more than 700 pages.
- Are the events X=’Being Included in The Worst Books of All Time list’ and Y=’Having more than 700 pages’ independent? Explain how you have obtained your answer.

###### Estimate the probability that a book has over 30% of the ratings above 4.
After inspecting the .json-file "lighter_authors" we decided to use the attribute "rate_dist". # We need to check if every document has such an attribute.
First we took a look at the format of this attribute by the following code:

In [3]:
# filename = "lighter_authors"
filename = "lighter_books"

with open('config.json') as cfg:
    cfg = json.load(cfg)
    path = os.path.join(cfg["datadir"], filename + '.' + "json")

    with open(path, 'r') as f:
        for i in range(10):  # read first 10 lines
            line = next(f).strip()
            print(line)
            json_obj = json.loads(line)
            # print(json_obj)

{"id":2,"title":"Harry Potter and the Order of the Phoenix (Harry Potter, #5)","authors":[{"id":"1077326","name":"J.K. Rowling","role":""},{"id":"2927","name":"Mary GrandPr\u00e9","role":"Illustrator"}],"author_name":"J.K. Rowling","author_id":1077326,"work_id":2809203,"isbn":"0439358078","isbn13":"9780439358071","asin":"","language":"eng","average_rating":4.5,"rating_dist":"5:1674064|4:664833|3:231195|2:41699|1:16215|total:2628006","ratings_count":2628006,"text_reviews_count":44716,"publication_date":"2004-09","original_publication_date":"2003-06-21","format":"Paperback","edition_information":"US Edition","image_url":"https:\/\/i.gr-assets.com\/images\/S\/compressed.photo.goodreads.com\/books\/1546910265l\/2._SX98_.jpg","publisher":"Scholastic Inc.","num_pages":870,"series_id":"45175","series_name":"Harry Potter","series_position":"5","shelves":[{"name":"to-read","count":324191},{"name":"currently-reading","count":52675},{"name":"fantasy","count":48251},{"name":"favorites","count":383

Seeing that the attribute looks in general like this example:

'rating_dist': '5:1674064|4:664833|3:231195|2:41699|1:16215|total:2628006'

Just with different values, we build our functions to process this information.

In [4]:
def parseToDict(mystr):
    myli = mystr.split("|")
    mydict = {}
    for entry in myli:
        k,v = entry.split(":")
        try:
            # k = int(k)
            v = int(v)
        except ValueError as e:
            print(e)
            continue
        mydict[k] = v
        # from collections import Counter
    
    return mydict

my_str = '5:1674064|4:664833|3:231195|2:41699|1:16215|total:2628006'

temp = parseToDict('5:1674064|4:664833|3:231195|2:41699|1:16215|total:2628006')
temp

{'5': 1674064,
 '4': 664833,
 '3': 231195,
 '2': 41699,
 '1': 16215,
 'total': 2628006}

In [7]:
def getPercentage(my_dict, key_numerator, key_denominator):
    try:
        return my_dict[key_numerator]/my_dict[key_denominator] # we interpret above 4 as strictly above 4.
    except ZeroDivisionError as e:
        print(e)
        return -1


getPercentage(temp, '5', "total")

0.6370092001312021

In [18]:
# load data
# filename = "lighter_authors"
filename = "lighter_books"

books_with_30Percent_of_ratings_above_4 = 0
books_with_at_least_one_rating = 0
# load config
with open('config.json') as cfg:
    cfg = json.load(cfg)
    path = os.path.join(cfg["datadir"], filename + '.' + "json")
    print(path)
    processed_rows = 0

    chunks = pd.read_json(path, lines=True, chunksize = 10**4, nrows = None)
    for chunk in chunks:
        chunk = chunk[["id","ratings_count", "rating_dist"]] # to shrink needed memory
        chunk = chunk[chunk["ratings_count"]>0] # to exclude books without ratings
        books_with_at_least_one_rating += len(chunk)
        chunk["percentage"] = chunk.apply(lambda row : getPercentage(parseToDict(row['rating_dist']),'5','total'), axis=1)        
        books_with_30Percent_of_ratings_above_4 += len(chunk[chunk["percentage"] > 0.3]) # we interpret over as strictly over
        # print(chunk)
        processed_rows += len(chunk)
        print(processed_rows) 
        # break
print(books_with_30Percent_of_ratings_above_4)



F:/adm/lighter_books.json
9882
19677
29454
39259
49043
58823
68566
78279
87994
97661
107307
116974
126627
136285
145958
155598
165122
174706
184295
193850
203418
213007
222555
232028
241579
251224
260764
270306
279858
289359
298908
308363
317779
327174
336620
345943
355590
365237
374896
384549
394225
403852
413462
422993
432528
442106
451615
461089
470621
480114
489552
499124
508640
518083
527535
537039
546392
555753
565204
574728
584272
593722
603127
612484
621734
630969
640299
649552
658829
668055
677278
686680
696122
705487
714734
724044
733412
742601
751834
761040
770353
779563
788718
797875
807119
816309
825526
834690
843781
852898
861825
870931
880048
889138
898172
907303
916258
925370
934305
943305
952363
961481
970440
979436
988577
997512
1006589
1015579
1024447
1033525
1042486
1051415
1060490
1069482
1078410
1087371
1096263
1105021
1113392
1121409
1129437
1137461
1145351
1153305
1161159
1169099
1177156
1184974
1192706
1200154
1207613
1215293
1222889
1230782
1238603
1246624
125

In [19]:
books_with_30Percent_of_ratings_above_4/books_with_at_least_one_rating

0.5221711201578421

Hence the estimated probability that a book has over 30% of the ratings above 4 is 0.5221711201578421.

In [15]:
myhead = pd.read_json( "F:/adm/hw01_lighter_updated/lighter_books_updated.json", lines=True ,nrows= 10**5)
# myhead

In [12]:
myhead["percentage"] = myhead.apply(lambda row : getPercentage(parseToDict(row['rating_dist']),'5','total'), axis=1)
myhead

,id,title,authors,author_name,author_id,work_id,isbn,isbn13,asin,language,average_rating,rating_dist,ratings_count,text_reviews_count,publication_date,original_publication_date,format,edition_information,image_url,publisher,num_pages,series_id,series_name,series_position,shelves,description,percentage
0,2,Harry Potter and the Order of the Phoenix (Har...,"[{'id': '1077326', 'name': 'J.K. Rowling', 'ro...",J.K. Rowling,1077326,2809203,0439358078,9780439358071,,eng,4.50,5:1674064|4:664833|3:231195|2:41699|1:16215|to...,2628006,44716,2004-09,2003-06-21,Paperback,US Edition,https://i.gr-assets.com/images/S/compressed.ph...,Scholastic Inc.,870,45175,Harry Potter,5,"[{'name': 'to-read', 'count': 324191}, {'name'...",There is a door at the end of a silent corrido...,0.637009
1,3,Harry Potter and the Sorcerer's Stone (Harry P...,"[{'id': '1077326', 'name': 'J.K. Rowling', 'ro...",J.K. Rowling,1077326,4640799,,,,eng,4.48,5:4801606|4:1681521|3:623286|2:145898|1:125040...,7377351,116930,2003-11-01,1997-06-26,Hardcover,Library Edition,https://i.gr-assets.com/images/S/compressed.ph...,Scholastic Inc,309,45175,Harry Potter,1,"[{'name': 'fantasy', 'count': 63540}, {'name':...",Harry Potter's life is miserable. His parents ...,0.650858
2,4,Harry Potter and the Chamber of Secrets (Harry...,None,J.K. Rowling,1077326,6231171,0439554896,9780439554893,,eng,4.43,5:1690166|4:781011|3:313727|2:54687|1:15453|to...,2855044,55286,2003-11-01,1998-07-02,Hardcover,,https://i.gr-assets.com/images/S/compressed.ph...,Scholastic,352,45175,Harry Potter,2,"[{'name': 'to-read', 'count': 282341}, {'name'...",The Dursleys were so mean and hideous that sum...,0.591993
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,"[{'id': '1077326', 'name': 'J.K. Rowling', 'ro...",J.K. Rowling,1077326,2402163,043965548X,9780439655484,,eng,4.57,5:1994597|4:696545|3:212678|2:28915|1:13959|to...,2946694,58023,2004-05-01,1999-07-08,Mass Market Paperback,,https://i.gr-assets.com/images/S/compressed.ph...,Scholastic Inc.,435,45175,Harry Potter,3,"[{'name': 'to-read', 'count': 292815}, {'name'...","For twelve long years, the dread fortress of A...",0.676893
4,6,Harry Potter and the Goblet of Fire (Harry Pot...,"[{'id': '1077326', 'name': 'J.K. Rowling', 'ro...",J.K. Rowling,1077326,3046572,,,,eng,4.56,5:1808039|4:663849|3:193604|2:27759|1:12425|to...,2705676,48637,2002-09-28,2000-07-08,Paperback,First Scholastic Trade Paperback Edition,https://i.gr-assets.com/images/S/compressed.ph...,Scholastic,734,45175,Harry Potter,4,"[{'name': 'to-read', 'count': 287086}, {'name'...",Harry Potter is midway through his training as...,0.668239
5,7,"The Harry Potter Collection (Harry Potter, #1-6)","[{'id': '1077326', 'name': 'J.K. Rowling', 'ro...",J.K. Rowling,1077326,21457570,0439887453,9780439887458,,eng,4.73,5:25063|4:4467|3:1103|2:227|1:282|total:31142,31142,975,2006-09-01,2005-01-01,Paperback,Box Set,https://i.gr-assets.com/images/S/compressed.ph...,Scholastic,,45175,Harry Potter,1-6,"[{'name': 'to-read', 'count': 5809}, {'name': ...","<div>Six years of magic, adventure, and myster...",0.804797
6,8,"Harry Potter Boxed Set, Books 1-5 (Harry Potte...","[{'id': '1077326', 'name': 'J.K. Rowling', 'ro...",J.K. Rowling,1077326,21457576,0439682584,9780439682589,,eng,4.79,5:49438|4:6112|3:1525|2:354|1:534|total:57963,57963,183,2004-09-13,2003-10-01,Paperback,,https://s.gr-assets.com/assets/nophoto/book/11...,Scholastic,2690,,,,"[{'name': 'to-read', 'count': 7030}, {'name': ...",Box Set containing Harry Potter and the Sorcer...,0.852923
7,10,"Harry Potter Collection (Harry Potter, #1-6)",None,J.K. Rowling,1077326,21457570,0439827604,9780439827607,,eng,4.73,5:25063|4:4467|3:1103|2:227|1:282|total:31142,31142,975,2005-09-12,2005-01-01,Hardcover,Box Set,https://i.gr-assets.com/images/S/compressed.ph...,Scholastic,3342,45175,Harry Potter,1-6,"[{'name': 'to-read', 'count': 5809}, {'name': ...","Six years of magic, adventure, and mystery mak...",0.804797
8,11,The Hitchhiker's Guide to the Galaxy (Hitchhik